In [1]:
search_for = 138
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.14241266250610352
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 1097396311
type: [1, 1, 1, 1, 138] 138
cases of this type: 362673936
100000 67.13350996558681
200000 120.7173156779574
300000 115.93282912981678
400000 108.85978764568002
500000 106.50111335802173
600000 108.11170609483133
700000 112.69113859645961
800000 106.17132277107837
900000 105.34467293676255
1000000 104.4090015149673
1100000 104.95819821256362
1200000 103.76972081723518
1300000 85.00133896751251
1400000 41.888867518232566
1500000 104.13898506654918
1600000 102.07072500631355
1700000 101.61179472041964
1800000 99.90219727048358
1900000 104.2610935810253
2000000 106.55215609369732
2100000 105.49579390658505
2200000 103.09188207320956
2300000 103.18155820460572
2400000 104.81045913251658
2500000 104.34570125315706
2600000 100.98184573693469
2700000 25.151834273363132
2800000 101.04048784027849
2900000 102.88288130277513
300

29300000 115.61218110288775
29400000 117.62776534081874
29500000 115.63423883789851
29600000 115.17403434746856
29700000 117.44000085844891
29800000 114.38382354278471
29900000 118.20208812796265
30000000 112.7070272800776
30100000 111.80491374229548
30200000 95.3340229257864
30300000 36.82673737889248
30400000 106.51509038212406
30500000 112.11198531876413
30600000 110.34638013096253
30700000 116.55006414474006
30800000 109.9511607568353
30900000 111.21419109365118
31000000 115.12055037902816
31100000 116.91884384992626
31200000 114.8689424894921
31300000 107.18202607874619
31400000 47.18711955070488
31500000 115.3111756862904
31600000 115.81769536235402
31700000 113.22172483235644
31800000 52.05173325366053
31900000 95.3252021102383
32000000 108.21593592275497
32100000 112.21207591048841
32200000 109.74247493142406
32300000 107.2015803327293
32400000 107.01212025298149
32500000 108.07153394011497
32600000 106.05362638485688
32700000 104.7873531899685
32800000 103.4838056920824
329000

58900000 97.81090916207467
59000000 99.48975195767643
59100000 94.36094009587646
59200000 24.510853488083672
59300000 101.25521197373338
59400000 99.48089115574751
59500000 98.3020352596011
59600000 99.97037940965267
59700000 102.79322651151152
59800000 102.2841300481497
59900000 105.23080080907509
60000000 103.13548484693564
60100000 44.892025342839865
60200000 109.29615542017523
60300000 107.16304324256767
60400000 107.6292761489069
60500000 109.40332022662487
60600000 115.16836982424604
60700000 111.92399426068596
60800000 110.47568121052878
60900000 67.40371611696305
61000000 81.00389263724394
61100000 111.81551161273883
61200000 107.62352134521566
61300000 107.07429261216232
61400000 114.92339367839453
61500000 101.09841394641718
61600000 99.66353483031112
61700000 98.3011576914556
61800000 34.34096046374657
61900000 90.39459248318904
62000000 101.4363087200201
62100000 100.75586490792861
62200000 108.03304001345722
62300000 103.17739861700733
62400000 104.09761715688663
62500000 

88800000 42.5319554390921
88900000 105.64426391659258
89000000 104.39751866639175
89100000 107.83962524105388
89200000 105.64653856951412
89300000 109.88571860083232
89400000 111.59574856767158
89500000 106.89556794027538
89600000 101.31165791251898
89700000 100.7498555656036
89800000 102.10479919942256
89900000 98.53191601293751
90000000 74.60202942455858
90100000 54.0494106922186
90200000 90.11277240903883
90300000 89.31866227373645
90400000 88.72906113392753
90500000 92.65722491260779
90600000 89.59857788450915
90700000 39.25830117600561
90800000 76.62992532694982
90900000 91.64965433331622
91000000 90.68300301737811
91100000 89.7141627893197
91200000 89.36697030522512
91300000 90.26662003360278
91400000 33.663886334497036
91500000 98.82781066896152
91600000 97.12910201154602
91700000 100.56224731018003
91800000 99.1638020320959
91900000 100.88890157651647
92000000 109.61738269469971
92100000 105.6684484924855
92200000 103.95181374796573
92300000 102.58213453764424
92400000 102.6405

118400000 101.39387734956841
118500000 100.23391991494056
118600000 98.62012567493156
118700000 96.20024401637497
118800000 101.25658678920021
118900000 91.54719425804261
119000000 92.83149070473792
119100000 81.61343221307625
119200000 39.33236735720022
119300000 86.98879333245365
119400000 92.38824424766598
119500000 89.39327017634072
119600000 45.86227289517781
119700000 68.30593521455356
119800000 89.52388979072282
119900000 89.38423793801832
120000000 90.03899849343922
120100000 33.250936979701024
120200000 91.20065595430852
120300000 90.59354529151369
120400000 93.5425188281982
120500000 96.35288165100138
120600000 95.22084248605725
120700000 102.63973351195531
120800000 99.07278870280415
120900000 103.03939089045234
121000000 92.42348075009382
121100000 94.9151144802391
121200000 97.1905150972469
121300000 96.71035060736665
121400000 89.58940447144717
121500000 90.53718367456376
121600000 86.59002278683536
121700000 85.79882307849363
121800000 42.8686385132944
121900000 72.36507

...

982900000 9.835049315931045
983000000 4.958959073471441
983100000 6.966094436544436
983200000 9.909295821810455
983300000 10.890519252810316
983400000 8.781886699819673
983500000 7.6647608261283615
983600000 6.85586954923008
983700000 1.924501526240446
983800000 7.471532282871478
983900000 7.628080833904947
984000000 8.265849977711643
984100000 7.001311163129253
984200000 6.550958083850616
984300000 3.7655253010865235
984400000 3.315612257465291
984500000 6.752859888079786
984600000 6.382365530722875
984700000 7.34404018391648
984800000 6.66334693858407
984900000 5.75799805942816
985000000 3.078935641361129
985100000 5.379868406286038
985200000 6.782746068381667
985300000 7.268750882767155
985400000 7.392421402866308
985500000 6.812018877657531
985600000 4.712249685241437
985700000 2.980721707576791
985800000 6.617522755361802
985900000 6.9484664129574885
986000000 7.532520567006457
986100000 6.543314106615481
986200000 5.478320137798578
986300000 3.898858551344668
986400000 5.24747316

1011200000 1.5853887654269074
1011300000 0.5005147419510372
1011400000 2.1037192613961855
1011500000 2.02092537458928
1011600000 2.2351505042659103
1011700000 2.063860332995407
1011800000 2.057602465519343
1011900000 1.0953499598157432
1012000000 1.3428116359370172
1012100000 2.07028026785932
1012200000 2.0542153709089
1012300000 2.2869545551522474
1012400000 1.715057319613961
1012500000 1.5433416272603453
1012600000 0.4981373337555945
1012700000 1.478163063180417
1012800000 1.7543745177667212
1012900000 1.9199192929429651
1013000000 1.6585494626976676
1013100000 1.7786447826320007
1013200000 0.9384760898508608
1013300000 0.6667481818236344
1013400000 1.7589350835937276
1013500000 1.5223270894181768
1013600000 1.7488718639510785
1013700000 1.3777794778036878
1013800000 1.3763266979108215
1013900000 0.4906770268866592
1014000000 1.0647759060892557
1014100000 1.3442121425222515
1014200000 1.8989397666049368
1014300000 1.471725779014072
1014400000 1.3330933484425864
1014500000 0.988841278

1036900000 1.1688698932167152
1037000000 1.6625987215848532
1037100000 2.069882537068238
1037200000 1.683416304860115
1037300000 1.6397535776515055
1037400000 1.907053354400211
1037500000 1.8754938432753783
1037600000 1.69913275649214
1037700000 1.842753197437869
1037800000 1.4624591631446877
1037900000 1.7574842460503863
1038000000 1.7002849088655896
1038100000 1.85509356299243
1038200000 1.7141098265978845
1038300000 1.6860328004600351
1038400000 1.4491836234515132
1038500000 1.721924383587359
1038600000 1.7554022383289776
1038700000 1.4998103120289406
1038800000 1.7422110632272065
1038900000 1.8337531641007223
1039000000 1.6086823862557948
1039100000 1.588025226351722
1039200000 1.979525965976305
1039300000 1.61900768003005
1039400000 1.3770331621762688
1039500000 2.007218128684191
1039600000 1.7314089354853512
1039700000 1.2414899217582789
1039800000 2.13218959912026
1039900000 1.8578510997359632
1040000000 1.5743524484309466
1040100000 2.334106787906111
1040200000 2.32647197776497

1058000000 0.9404097954670548
1058100000 1.0013660432244817
1058200000 1.2740968677619278
1058300000 0.8911150810245819
1058400000 0.8270442562803653
1058500000 1.4198112415874977
1058600000 0.9492036230854537
1058700000 0.8609155491389937
1058800000 1.010595067965889
1058900000 1.0010689260946122
1059000000 0.8632660590636094
1059100000 0.9433740150432666
1059200000 1.132017402153272
1059300000 0.7398330308713938
1059400000 0.8786523917301575
1059500000 1.1527507170727174
1059600000 0.9260598006640713
1059700000 0.7729449003477454
1059800000 1.1214773956116213
1059900000 0.957934538916001
1060000000 0.8666177937372088
1060100000 0.9583180687234932
1060200000 1.113460595609149
1060300000 0.7001533727778118
1060400000 0.9140552665644069
1060500000 1.1328353621204177
1060600000 0.8470231981651067
1060700000 0.7352123156596476
1060800000 1.0813051394093607
1060900000 0.8854915694825193
1061000000 0.6866911500509362
1061100000 1.0374017648038918
1061200000 1.0407371342900553
1061300000 0.7

1084500000 0.09019112470927636
1084600000 0.0581107227946361
1084700000 0.057719925814136534
best so far: 0
type: [2, 1, 3, 1, 23] 138
cases of this type: 2518569
1084800000 0.139483580383973
1084900000 0.21354063664877415
1085000000 0.19169109266799222
1085100000 0.1765869402758327
1085200000 0.17339939447827538
1085300000 0.17409540050180952
1085400000 0.17810804296849173
1085500000 0.17637862730522155
1085600000 0.17478345690007543
1085700000 0.18374529122732147
1085800000 0.10768370106412767
1085900000 0.0598922735710515
1086000000 0.05927700587729746
1086100000 0.05879173393704692
1086200000 0.05871313705116643
1086300000 0.07679359953537253
1086400000 0.2007056138032655
1086500000 0.16575364463552966
1086600000 0.16007103047297333
1086700000 0.15722762687357067
1086800000 0.15202607662720746
1086900000 0.1528093902963155
1087000000 0.1481876721912384
1087100000 0.14856315658240252
1087200000 0.16236874475152163
best so far: 0
type: [2, 1, 3, 23, 1] 138
cases of this type: 109503
